In [36]:
import numpy as np
import pandas as pd

request_json_27179 = {'item_id': '27179', 'init_inventory': 149, 'min_order_qty': 50, 'lead_time': 16, 'order_unit': 10, 
                      'demand': [6.0, 8.0, 9.0, 8.0, 9.0, 6.0, 4.0, 6.0, 8.0, 9.0, 8.0, 9.0, 6.0, 4.0, 6.0, 8.0, 9.0, 8.0, 9.0, 6.0, 4.0, 6.0, 8.0, 9.0, 8.0, 9.0, 6.0, 4.0, 6.0, 8.0, 9.0, 8.0, 9.0, 6.0, 4.0, 6.0, 8.0, 9.0, 8.0, 9.0, 6.0, 4.0]}

request_json = {'item_id': '10961', 'init_inventory': 87, 'min_order_qty': 10, 'lead_time': 6, 'order_unit': 5,
                'demand':[5.3, 4.6, 3.8, 4.7, 5.4, 6.8, 2.9, 19.1, 27.4, 34.6, 22.1, 30.8, 20.4, 14.8, 9.3, 14.7, 20.0, 13.5, 17.8, 10.5, 8.5, 6.9, 7.5, 9.0, 7.8, 6.1, 7.4, 5.5, 4.0, 7.4]}

In [37]:
'''
for cloud function
Bianca ZYCao
'''
def smart_replenisher(request_json):
    from ortools.linear_solver import pywraplp
    # Create the MIP solver.
    solver = pywraplp.Solver.CreateSolver('SCIP')

    # Define the input data. 
    init_inventory_level = request_json['init_inventory']
    big_M = 2000
    demand = [request_json['demand']]
    num_of_demand_scenarios = len(demand)
    num_of_weeks = len(demand[0]) 
    lead_time = request_json['lead_time']
    min_order_qty = request_json['min_order_qty']
    order_unit = request_json['order_unit']
    # TODO: introduce more constraints type & define objective function Ordering_cost+holding_cost+penalty on stockout
    ordering_cost = 20
    holding_cost = 0.2
    unit_price = 1
    service_rate = 0.98
    order_placement_interval = 3
    # Define the decision variables.
    order_quantity = [solver.IntVar(0, 100, f'order_quantity_{i}') 
                        for i in range(num_of_weeks)]
    inventory_level = [[solver.IntVar(-500, 2000,  f'inventory_week{i}_scenario{j}') 
                        for i in range(num_of_weeks+1)] 
                        for j in range(num_of_demand_scenarios)]
    total_inventory_level_scenario = [solver.IntVar(-50000, solver.infinity(),  
                                                    f'exp_inventory_level_scenario{j}') 
                        for j in range(num_of_demand_scenarios)]
    stock_state  = [[solver.IntVar(0, 1, f'stock_state_week{i}_scenario{j}')
                        for i in range(num_of_weeks+1)] 
                        for j in range(num_of_demand_scenarios)]
    stock_state_sum = [solver.IntVar(0, num_of_weeks, f'stock_state_sum_scenario{j}') 
                        for j in range(num_of_demand_scenarios)]

    # Add constraint for each week's inventory level
    for j in range(num_of_demand_scenarios):
        solver.Add(inventory_level[j][0] == init_inventory_level)  # initial inventory level
        for i in range(num_of_weeks):
            if i % order_placement_interval != 0: 
                solver.Add(order_quantity[i] == 0)
            inventory_level[j][i+1] = inventory_level[j][i] - demand[j][i]
            if i >= lead_time - 1 :
                inventory_level[j][i+1] += order_quantity[i-(lead_time - 1)]*order_unit
            # add constraint to tell stock out or not
            solver.Add(inventory_level[j][i] <= stock_state[j][i] * big_M )
            solver.Add(inventory_level[j][i] >= 1/big_M - (1-stock_state[j][i]) * big_M )
        stock_state_sum[j] = sum(stock_state[j])
        total_inventory_level_scenario[j] = sum(inventory_level[j])
        
    # good case
    solver.Add(sum(stock_state_sum)>=service_rate*num_of_weeks*num_of_demand_scenarios)

    # Define the objective function to minimize inventory level.
    accum_inventory_level = sum(total_inventory_level_scenario)/num_of_demand_scenarios
    holding_value = unit_price * accum_inventory_level
    total_cost = ordering_cost * 3 + holding_value + holding_cost * holding_value
    solver.Minimize(accum_inventory_level)

    # Solve the optimization problem.
    status = solver.Solve()

    # Print the optimal order quantity and total cost.
    if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
        print('Optimal or Feasible solution found.')
        print('Optimal order quantity:')
        for i in range(num_of_weeks):
            print(order_quantity[i].solution_value()*order_unit,end=', ' )
        print('')
        service_rate = sum(stock_state_sum[j].solution_value() for j in range(num_of_demand_scenarios))/(num_of_weeks*num_of_demand_scenarios) 
        avg_inventory_level = sum(total_inventory_level_scenario[j].solution_value() for j in range(num_of_demand_scenarios))/(num_of_weeks*num_of_demand_scenarios)
        exp_inventory_level = []
        for i in range(num_of_weeks):
            exp_inventory_level.append(sum(inventory_level[j][i].solution_value() for j in range(num_of_demand_scenarios)))
        # return {
        #     'message': 'Optimal or Feasible solution found.',
        #     'service rate: ': service_rate,
        #     'AVG inventory level:': avg_inventory_level,
        #     'Holding Value :': holding_value.solution_value(),
        #     'Total Cost:': total_cost.solution_value(),
        # }
        return {
            'solution_found': True,
            'replenish_required_smart': order_quantity[0].solution_value()>0,
            'suggested_qty_smart:': max(order_quantity[0].solution_value()*order_unit,min_order_qty) if order_quantity[0].solution_value()>0 else 0,
        }
    elif  status == pywraplp.Solver.INFEASIBLE:
        return {'solution_found': False,'message': 'The problem is not feasible.'}
    elif  status == pywraplp.Solver.MODEL_INVALID:
        raise RuntimeError('The model is not valid.')
    else:
        return {'solution_found': False,
                'message': 'The problem does not have an optimal solution.'}
    
    return 'Nothing!'

### try a few SKUs

In [38]:
request_json = {'item_id': '6112', 'init_inventory': 125, 'min_order_qty': 30, 'lead_time': 25, 'order_unit': 10, 
                'demand': [8.0, 6.0, 4.0, 10.0, 3.0, 6.0, 1.0, 8.0, 6.0, 4.0, 10.0, 3.0, 6.0, 1.0, 8.0, 6.0, 4.0, 10.0, 3.0, 6.0, 1.0, 8.0, 6.0, 4.0, 10.0, 3.0, 6.0, 1.0, 8.0, 6.0, 4.0, 10.0, 3.0, 6.0, 1.0, 8.0, 6.0, 4.0, 10.0, 3.0, 6.0, 1.0, 8.0, 6.0, 4.0, 10.0, 3.0, 6.0, 1.0, 8.0, 6.0, 4.0, 10.0, 3.0, 6.0, 1.0]}

smart_replenisher(request_json)

Optimal or Feasible solution found.
Optimal order quantity:
30.0, 0.0, 0.0, 20.0, 0.0, 0.0, 10.0, 0.0, 0.0, 20.0, 0.0, 0.0, 20.0, 0.0, 0.0, 10.0, 0.0, 0.0, 20.0, 0.0, 0.0, 20.0, 0.0, 0.0, 10.0, 0.0, 0.0, 20.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 


{'solution_found': True,
 'replenish_required_smart': True,
 'suggested_qty_smart:': 30.0}

In [39]:
request_json_27179 = {'item_id': '27179', 'init_inventory': 108, 'min_order_qty': 50, 'lead_time': 16, 'order_unit': 10, 
                      'demand': [6.0, 8.0, 9.0, 8.0, 9.0, 6.0, 4.0, 6.0, 8.0, 9.0, 8.0, 9.0, 6.0, 4.0, 6.0, 8.0, 9.0, 8.0, 9.0, 6.0, 4.0, 6.0, 8.0, 9.0, 8.0, 9.0, 6.0, 4.0, 6.0, 8.0, 9.0, 8.0, 9.0, 6.0, 4.0, 6.0, 8.0, 9.0, 8.0, 9.0, 6.0, 4.0]}

smart_replenisher(request_json_27179)

Optimal or Feasible solution found.
Optimal order quantity:
20.0, 0.0, 0.0, 30.0, 0.0, 0.0, 20.0, 0.0, 0.0, 20.0, 0.0, 0.0, 20.0, 0.0, 0.0, 30.0, 0.0, 0.0, 10.0, 0.0, 0.0, 30.0, 0.0, 0.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 


{'solution_found': True,
 'replenish_required_smart': True,
 'suggested_qty_smart:': 50}

In [40]:
request_json_395 = {'item_id': '395', 'init_inventory': 333, 'min_order_qty': 10, 'lead_time': 26, 'order_unit': 5, 'demand': [9.0, 15.0, 20.0, 14.0, 18.0, 10.0, 8.0, 9.0, 15.0, 20.0, 14.0, 18.0, 10.0, 8.0, 9.0, 15.0, 20.0, 14.0, 18.0, 10.0, 8.0, 9.0, 15.0, 20.0, 14.0, 18.0, 10.0, 8.0, 9.0, 15.0, 20.0, 14.0, 18.0, 10.0, 8.0, 9.0, 15.0, 20.0, 14.0, 18.0, 10.0, 8.0, 9.0, 15.0, 20.0, 14.0, 18.0, 10.0, 8.0, 9.0, 15.0, 20.0, 14.0, 18.0, 10.0, 8.0]}

smart_replenisher(request_json_395)

Optimal or Feasible solution found.
Optimal order quantity:
45.0, 0.0, 0.0, 25.0, 0.0, 0.0, 60.0, 0.0, 0.0, 35.0, 0.0, 0.0, 50.0, 0.0, 0.0, 30.0, 0.0, 0.0, 45.0, 0.0, 0.0, 40.0, 0.0, 0.0, 40.0, 0.0, 0.0, 45.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 


{'solution_found': True,
 'replenish_required_smart': True,
 'suggested_qty_smart:': 45.0}